In [ ]:
# Here is a lot of unncesessary info presented, 
# main point is to show that pattern works across main categories also, 
# not only for high category level

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
import pandas as pd
from collections import Counter
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:

df = pd.read_csv(
    "/Users/macos/Documents/sentiment_project/validation/main_category.csv"
)
df.groupby("main_category").size()

In [ ]:
def get_sample_sizes(n_total):
    if n_total <= 100:
        return [min(20, n_total // 2), n_total]
    elif n_total <= 300:
        return [50, 75, 100, 125, 150, 200, min(250, n_total)]
    elif n_total <= 800:
        return [50, 100, 150, 200, 250, 300, 400, 500, min(700, n_total)]
    elif n_total <= 1500:
        return [100, 250, 500, 600, 650, 750, min(1000, n_total)]
    elif n_total <= 4000:
        return [100, 250, 500, 750, 1000, 1500, 2000, min(3000, n_total)]
    else:
        return [100, 250, 500, 750, 1000, 1500, 2000, 2500, 3000, 3500, 4000, n_total]


In [ ]:
def tokenize(text):
    """Simple tokenizer converting text to lowercase words."""
    return re.findall(r"\b\w+\b", str(text).lower())


# === Constants ===
TOP_N = 100
TARGET_COLUMN = "aspect_summary"

print(
    "\n🧠 WORD STABILITY ANALYSIS BY HIGH_CATEGORY → MAIN_CATEGORY (Progressive Sample Sizes)\n"
)
all_results = []

# === Iterate through each high_category ===
for high_cat, df_high in df.groupby("high_category"):
    print(f"\n🔹 HIGH CATEGORY: {high_cat}")

    # === Iterate through each main_category within this high_category ===
    for main_cat, df_main in df_high.groupby("main_category"):
        print(f"   ↳ MAIN CATEGORY: {main_cat}")
        n_total = len(df_main)
        SAMPLE_SIZES = get_sample_sizes(n_total)

        results = []
        prev_words = None
        prev_freqs = None

        for n in SAMPLE_SIZES:
            if len(df_main) < n:
                continue  # skip if not enough data

            sample = df_main.iloc[:n]

            # --- Tokenize and count words ---
            words = [w for text in sample[TARGET_COLUMN] for w in tokenize(text)]
            freqs = Counter(words)
            top_words = dict(freqs.most_common(TOP_N))

            # --- Compare with previous sample ---
            if prev_words is not None:
                # 1️⃣ Overlap (Jaccard similarity)
                overlap = len(set(prev_words) & set(top_words)) / len(
                    set(prev_words) | set(top_words)
                )

                # 2️⃣ Average frequency change
                common_words = set(prev_words) & set(top_words)
                changes = []
                for w in common_words:
                    prev_count = prev_freqs[w]
                    curr_count = top_words[w]
                    rel_change = abs(curr_count - prev_count) / max(prev_count, 1)
                    changes.append(rel_change)
                avg_change = np.mean(changes) if changes else 0

                results.append(
                    {
                        "sample_size": n,
                        "word_overlap_%": round(overlap * 100, 2),
                        "avg_freq_change_%": round(avg_change * 100, 2),
                    }
                )

            prev_words = top_words
            prev_freqs = freqs

        # === Results per main_category ===
        if results:
            results_df = pd.DataFrame(results)
            print(results_df.to_string(index=False))

            # store all results together
            for row in results:
                all_results.append(
                    {"high_category": high_cat, "main_category": main_cat, **row}
                )

# === Global summary ===
print("\n📘 INTERPRETATION:")
print(
    "• Each table shows how stable top vocabulary becomes as sample size grows within each main_category."
)
print(
    "• 'word_overlap_%' → similarity of top words between consecutive samples (higher = more stable)."
)
print(
    "• 'avg_freq_change_%' → how much word frequencies fluctuate across sizes (lower = more stable)."
)
print(
    "• Expect overlap to increase and freq change to flatten as the data stabilizes.\n"
)